# Czyszczenie danych i przygotowanie zbioru do analizy
## Autor: Mateusz Jagoda


## Opis problemu i cel 
Proces przygotowania danych jest absolutnie kluczowym procesem w data miningu. Dane, które otrzymujemy na przykład do uczenia maszynowego mogą mieć bardzo wiele problemów, takich jak pochodzenie z różnych niespójnych ze sobą źródeł, zawieranie błędów i powtórzeń, format nienadający się do dalszej pracy na nich lub nadmiarowe parametry. Zadaniem osoby przeprowadzającej proces przygotowania danych jest rozpoznanie i wyeliminowanie tych problemów.

W niniejszym dokumencie przeprowadzony został proces czyszczenia danych dla zbioru TitanicMess. Zbiór ten zawiera dane o ofiarach i ocalałych katastrofy Titanica. Przeprowadzono analizę identyfikującą problemy zbioru danych, a następnie podjęto się rozwiązania ich.

## Identyfikacja problemów


Po załadowaniu pliku wyświetlono data frame zawierający dane. 

In [857]:
import pandas as pd  
import numpy as np
pd.options.mode.chained_assignment = None
df = pd.read_csv('TitanicMess.tsv',sep='\t',encoding = "UTF-8", header=0)
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ship
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,"7,25",NaN,S,Titanic
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,"71,2833",C85,C,Titanic
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,"7,925",NaN,S,Titanic
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,"53,1",C123,S,Titanic
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,"8,05",NaN,S,Titanic
...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,888,1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30,B42,S,Titanic
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,"23,45",NaN,S,Titanic
889,890,1,1,"Behr, Mr. Karl Howell",male,26,0,0,111369,30,C148,C,Titanic
890,891,0,3,"Dooley, Mr. Patrick",male,32,0,0,370376,"7,75",NaN,Q,Titanic


Już na tym etapie zauważono pierwsze problemy - **kolumna "ship" jako jedyna nazwana jest z małej litery**, natomiast **kolumny "SibSp" oraz "Parch" są nazwane z sposób niezrozumiały dla osoby niezapoznanej z tym zbiorem danych**. Zauważono również, że w niektórych kolumnach występują wartości "NaN'.
W celu sprawdzenia powagi problemu wartości pustych zliczono je dla każdej z kolumn.

In [858]:
df.isnull().sum() 

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            173
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          685
Embarked         2
ship             0
dtype: int64

Wartości "NaN" wystąpiły w kolumnach takich jak "Age", "Cabin" i "Embarked". Ilość braków w kolumnie "Cabin" jest na tyle duża, że można uznać, że nie o wszystkich pasażerach dostępne były tak szczegółowe dane. **Braki w kolumnach "Age" oraz "Embarked" natomiast powinny zostać naprawione.**

W następnym kroku zliczono wartości unikalne w każdej z kolumn.

In [859]:
df.nunique()

PassengerId    888
Survived         2
Pclass           3
Name           889
Sex              6
Age             93
SibSp            7
Parch            6
Ticket         680
Fare           250
Cabin          145
Embarked         6
ship             1
dtype: int64

Zabieg ten pomógł zlokalizować szereg problemów:  
**-unikalnych wartości w kolumnie "PassengerId" jest mniej niż wszystkich wierszy - oznacza to, że występują tam jakieś duplikaty**  
**-w kolumnie "Sex" znajduje się 6 różnych wartości, a powinny być 2**  
**-w kolumnie "Embarked" znajduje się 6 różnych wartości, a powinny być 3 (na Titanic pasażerowie wsiadali w 3 różnych portach)**  
**-w kolumnie "ship" we wszystkich wierszach występuje ta sama wartość**  

Postanowiono zdobyć więcej informacji dotyczących każdego z tych problemów. W związku z tym zlokalizowano duplikaty w kolumnie "PassengerId"

In [860]:
df[df.duplicated(['PassengerId'], keep=False)]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ship
10,11,1,3,"Sandstrom, Miss. Marguerite Ru&5$$",female,4,1,1,PP 9549,"16,7",G6,S,Titanic
13,11,1,3,"Sandstrom, Miss. Marguerite Ru&5$$",female,4,1,1,PP 9549,"16,7",G6,S,Titanic
23,11,1,3,"Sandstrom, Miss. Marguerite Ru&5$$",female,4,1,1,PP 9549,"16,7",G6,S,Titanic
224,225,1,1,"Hoyt, Mr. Frederick Maxfield",male,38,1,0,19943,90,C93,S,Titanic
520,225,1,1,"Hoyt, Mr. Frederick Maxfield",male,38,1,0,19943,90,C93,S,Titanic
678,225,1,1,"Hoytt, Mr. Frederick Maxfield",male,38,1,0,19943,90,C93,S,Titanic


oraz wypisano wartości występujące w kolumnach "Sex", "Embarked" i "ship" wraz z ilością wystąpień.

In [861]:
df['Sex'].value_counts()

male       576
female     312
malef        1
femmale      1
fem          1
mal          1
Name: Sex, dtype: int64

In [862]:
df['Embarked'].value_counts()

S     643
C     167
Q      76
So      2
Qe      1
Co      1
Name: Embarked, dtype: int64

In [863]:
df['ship'].value_counts()

Titanic    892
Name: ship, dtype: int64

Jako ostatni krok identyfikacji problemów sprawdzono, czy w poszczególnych kolumnach zawierających wartości liczbowe nie występują "dziwne" lub mocno odstające wartości.W tym celu wypisano typy wszystkich kolumn, zwracając uwagę czy kolumny "Pclass", "Age", "SibSp", "Parch", "Fare" mają wartości liczbowe.

In [864]:
df.dtypes

PassengerId     int64
Survived        int64
Pclass          int64
Name           object
Sex            object
Age            object
SibSp           int64
Parch           int64
Ticket         object
Fare           object
Cabin          object
Embarked       object
ship           object
dtype: object

Zabieg ten pokazał, że kolumny takie jak **"Age" oraz "Fare" nie są typu liczbowego.** Na tym etapie uniemożliwia to głębszą analizę wartości, jakie w nich występują - zostanie ona wykonana w następnym rozdziale, po przekonwertowaniu wartości tych kolumn na liczbowe. Dla kolumn "Pclass", "SibSp" oraz "Parch" wypisano wartości minimalne i maksymalne.

In [865]:
def getColumnMinAndMaxText(columnName):
    return columnName + '\n' + "max: " + str(df[columnName].dropna().max()) + '\n' + "min: " + str(df[columnName].dropna().min()) +'\n'

print(getColumnMinAndMaxText('Pclass'))
print(getColumnMinAndMaxText('SibSp'))
print(getColumnMinAndMaxText('Parch'))

Pclass
max: 3
min: 1

SibSp
max: 8
min: 0

Parch
max: 5
min: 0



Wartości te mieszczą się w oczekiwanym zakresie.

## Rozwiązanie problemów

Pierwszym z rozwiązanych problemów były błędy w nazewnictwie kolumn. Nazwę kolumny "ship" zmieniono na "Ship", a kolumny "SibSp" i "Parch" złączono w jedną nazwaną "NoOfCoTravellers"

In [866]:
df.rename(columns={'ship':'Ship'}, inplace=True)
sum_column = df['SibSp'] + df['Parch']
df['NoOfCoTravellers'] = sum_column.values
del df['SibSp']
del df['Parch']
df

,PassengerId,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked,Ship,NoOfCoTravellers
0,1,0,3,"Braund, Mr. Owen Harris",male,22,A/5 21171,"7,25",NaN,S,Titanic,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,PC 17599,"71,2833",C85,C,Titanic,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26,STON/O2. 3101282,"7,925",NaN,S,Titanic,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,113803,"53,1",C123,S,Titanic,1
4,5,0,3,"Allen, Mr. William Henry",male,35,373450,"8,05",NaN,S,Titanic,0
...,...,...,...,...,...,...,...,...,...,...,...,...
887,888,1,1,"Graham, Miss. Margaret Edith",female,19,112053,30,B42,S,Titanic,0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,W./C. 6607,"23,45",NaN,S,Titanic,3
889,890,1,1,"Behr, Mr. Karl Howell",male,26,111369,30,C148,C,Titanic,0
890,891,0,3,"Dooley, Mr. Patrick",male,32,370376,"7,75",NaN,Q,Titanic,0


Następnie usunięto wiersze, w których wartości puste znalazły się w kolumnach "Age" lub "Embarked".

In [867]:
df = df[df['Age'].notna()]
df = df[df['Embarked'].notna()]

W kolejnym kroku, bazując na rozpoznanym wcześniej problemie z powtarzającymi się wartościami w kolumnie "PassengerId", pozbyto się wpisów będących duplikatami.

In [868]:
df.drop_duplicates(inplace = True)
df

,PassengerId,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked,Ship,NoOfCoTravellers
0,1,0,3,"Braund, Mr. Owen Harris",male,22,A/5 21171,"7,25",NaN,S,Titanic,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,PC 17599,"71,2833",C85,C,Titanic,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26,STON/O2. 3101282,"7,925",NaN,S,Titanic,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,113803,"53,1",C123,S,Titanic,1
4,5,0,3,"Allen, Mr. William Henry",male,35,373450,"8,05",NaN,S,Titanic,0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27,211536,13,NaN,S,Titanic,0
887,888,1,1,"Graham, Miss. Margaret Edith",female,19,112053,30,B42,S,Titanic,0
889,890,1,1,"Behr, Mr. Karl Howell",male,26,111369,30,C148,C,Titanic,0
890,891,0,3,"Dooley, Mr. Patrick",male,32,370376,"7,75",NaN,Q,Titanic,0


Następnie rozwiązano problem z literówkami pojawiającymi się w kolumnie "Sex" i dwoma konwencjami nazewnictwa w kolumnie "Embarked".

In [869]:
df['Sex'] = df['Sex'].replace(['malef', 'mal'], 'male')
df['Sex'] = df['Sex'].replace(['femmale', 'fem'], 'female')
df['Embarked'] = df['Embarked'].replace('So', 'S')
df['Embarked'] = df['Embarked'].replace('Qe', 'Q')
df['Embarked'] = df['Embarked'].replace('Co', 'C')

Usunięto kolumnę "Ship" - mogłaby być użyteczna, gdyby zbiór został połączony z innym zbiorem, zawierającym dane z katastrofy innego statku, jednak w przypadku gdy wiemy, że w całym zbiorze jest mowa o Titanicu, jest ona zbędna.

In [870]:
del df['Ship']

Po czym przystąpiono do oczyszczenia kolumny "Age". Zmieniono jej typ na liczbowy, sprawdzono maksymalną i minimalną wartość i usunięto wszystkie wpisy spoza zakresu 0<Age<100

In [871]:
df['Age'] = df['Age'].str.replace(',', '.').astype(float)
print(getColumnMinAndMaxText('Age'))
df = df[df['Age'] > 0]
df = df[df['Age'] < 100] 

Age
max: 4435.0
min: -12.0



W ostatnim kroku przystąpiono do naprawienia kolumny "Fare". W jej przypadku przed zmianą typu na liczbowy należało pozbyć się wartości nie dających się przekonwertować na liczbę.

In [872]:
df['Fare'] = df['Fare'].str.replace(',', '.')
df = df[df.Fare.apply(lambda x: x.isnumeric())]
df['Fare'] = df['Fare'].astype(float)
print(getColumnMinAndMaxText('Fare'))

Fare
max: 263.0
min: 0.0



Maksymalna wartość w kolumnie "Fare" nie wzbudza żadnych podejrzeń. Najmniejsza, wynosząca 0, mogła by być uznana za błąd, jednak można założyć, że z jakiegoś powodu część biletów była darmowa - nie podjęto więc działań mających na celu zmianę tej wartości.

Oczyszczone dane zapisano do pliku

In [873]:
df.to_csv('TitanicCleaned.tsv', index=False, sep="\t")